# U-Net标签的制作
使用之前写过的KMeans聚类来辅助标注，将二聚类的结果以8位彩色png存储，方便U-Net训练

In [1]:
import cv2
from pylab import *


def img2array(img):
    img = cv2.resize(img, (300, 300))
    # 由BGR改为RGB防止颜色偏差
    img = img[:, :, [2, 1, 0]]
    features = list()
    for x in range(300):
        for y in range(300):
            features.append([img[x][y][0], img[x][y][1], img[x][y][2]])
    features = np.array(features, 'f')  # 变为数组
    return img, features

In [2]:
# 定义距离(使用RGB的值之间的距离)
def dist(vecA, vecB):
    sum = 0
    for i in range(3):
        if vecA[i] != vecA[i]:
            vecA[i] = 0
        if vecB[i] != vecB[i]:
            vecB[i] = 0
        sum += (int(vecA[i]) - int(vecB[i])) ** 2
    return sqrt(sum)

In [3]:
def rand_cent(data_mat, k):
    n = shape(data_mat)[1]  # 获取坐标维数
    centroids = mat(zeros((k, n)))  # k个n维的点
    for j in range(n):
        minJ = min(data_mat[:, j])  # 本维最小值
        rangeJ = float(max(data_mat[:, j]) - minJ)  # 本维的极差
        centroids[:, j] = mat(minJ + rangeJ * np.random.rand(k, 1))  # 随机值
    return centroids


#     print(centroids)

In [4]:
def kMeans(data_mat, k):
    m = shape(data_mat)[0]
    # 初始化点的簇
    cluster_assment = mat(zeros((m, 2)))  # 类别，距离
    # 随机初始化聚类初始点
    centroid = rand_cent(data_mat, k)
    cluster_changed = True
    # 遍历每个点
    while cluster_changed:  # 等到迭代不发生变化即停止
        cluster_changed = False
        for i in range(m):
            min_index = -1
            min_dist = inf
            for j in range(k):
                distance = dist(data_mat[i], np.array(centroid)[j])
                if distance < min_dist:
                    min_dist = distance
                    min_index = j
            if cluster_assment[i, 0] != min_index:
                cluster_changed = True
                cluster_assment[i, :] = min_index, min_dist ** 2
        # 计算簇中所有点的均值并重新将均值作为质心
        for j in range(k):
            per_data_set = data_mat[nonzero(cluster_assment[:, 0].A == j)[0]]
            centroid[j, :] = mean(per_data_set, axis=0)
        t=[0,0,0]
        if (sum(centroid[0,:])<sum(centroid[1,:]))and(sum(centroid[0,:])>300):
            for i in range(3):
                t[i] = centroid[0,i]
                centroid[0,i] = centroid[1,i]
                centroid[1,i] = t[i]
    return centroid, cluster_assment

以上是实验2时的KMeans聚类函数，用于辅助生成标签文件，接下来将标签存为png格式以便U-Net训练

In [5]:
from scipy.cluster.vq import *
from PIL import Image
import os


def kMeansSave(Img, filename):
    Img, Features = img2array(Img)
    # 聚类
    centroids, variance = kMeans(Features, 2)
    # 使用scipy.cluster.vq绘制聚类结果
    code, distance = vq(Features, np.nan_to_num(centroids))
    # 用聚类标记创建图像
    codeimg = code.reshape(224， 224)
    # np_array = np.zeros((224, 224, 3), dtype=np.uint8)
    image = Image.fromarray(codeimg)
    image.save(f'./TestDataSet(resized)/labels/{filename}.png')

In [ ]:
imgpath = 'D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/'
image_list = os.listdir(imgpath)
i= 0
for name in image_list:
#     i = i+ 1
#     if i>=1600:
        print(os.path.join(imgpath, name))
        img = cv2.imread(os.path.join(imgpath, name))
        filename = os.path.basename(name).split('.')[0]
        kMeansSave(img, filename)

D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1000).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1002).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1004).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1006).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1007).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1010).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1011).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1019).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1021).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1023).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1024).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1026).jpeg
D:/J

C:\Users\wang\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\wang\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (106).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1062).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1067).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1070).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1074).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1075).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (108).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1083).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1085).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1089).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (109).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1092).jpeg
D:/Jupy

D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1351).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1353).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1354).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1366).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1370).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1372).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1375).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (138).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1387).jpeg
D:/JupyterProject/ML_CurriculumDesign/TestDataSet(resized)/cloud/cloud (1389).jpeg
